# Combining Datasets

## Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.

The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

In [1]:
import pandas as pd
import numpy as np

In [2]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
    
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, 
                                              eval(a)._repr_html_())
                         for a in self.args)

## Relational Algebra

The behavior implemented in ``pd.merge()`` is a subset of what is known as *relational algebra*, which is a formal set of rules for manipulating relational data, and forms the conceptual foundation of operations available in most databases.
The strength of the relational algebra approach is that it proposes several primitive operations, which become the building blocks of more complicated operations on any dataset.
With this lexicon of fundamental operations implemented efficiently in a database or other program, a wide range of fairly complicated composite operations can be performed.

Pandas implements several of these fundamental building-blocks in the ``pd.merge()`` function and the related ``join()`` method of ``Series`` and ``Dataframe``s.
As we will see, these let you efficiently link data from different sources.

## Categories of Joins

 ``pd.merge()`` - function implements a number of types of joins: 

- *one-to-one* join  
- *many-to-one* join 
- *many-to-many* join 

All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the type of join performed depends on the form of the input data.

Here we will show simple examples of the three types of merges, and discuss detailed options further below.

### One-to-one joins

- `one-to-one join` -  is in many ways very similar to the column-wise concatenation

In [3]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']
                    })

df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]
                    })

In [4]:
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


To combine this information into a single ``DataFrame``, we can use the ``pd.merge()`` function:

In [5]:
df3 = pd.merge(df1, df2)# there is a common column employee to perform merge 
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


- The ``pd.merge()`` function recognizes that each ``DataFrame`` has an `employee` column, and automatically joins using this column as a key.
- The result of the merge is a new ``DataFrame`` that combines the information from the two inputs.

- the order of entries in each column is not necessarily maintained
    - in this case, the order of the "employee" column differs between ``df1`` and ``df2``, and the ``pd.merge()`` function correctly accounts for this.
- the merge in general discards the index, except in the special case of merges by index (see the ``left_index`` and ``right_index`` keywords, discussed momentarily).

### Many-to-one joins 

- Many-to-one joins are joins in which one of the two key columns contains duplicate entries.

- For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate. 

In [6]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']
                    })

In [7]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']
                    })

In [8]:
display('df4', 'df5', 'pd.merge(df3, df4)')

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


- The resulting ``DataFrame`` has an aditional column with the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

### Many-to-many joins 


- If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.


Consider the following, where we have a ``DataFrame`` showing one or more skills associated with a particular group.

By performing a many-to-many join, we can recover the skills associated with any individual person:

In [9]:
display('df1', 'df5', 'pd.merge(df1, df5)')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


- These three types of joins can be used with other Pandas tools to implement a wide array of functionality.
    - But in practice, datasets are rarely as clean as the one we're working with here.
    
In the following section we'll consider some of the options provided by ``pd.merge()`` that enable you to tune how the join operations work.

## Specification of the Merge Key

- often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

### The ``on`` keyword

We can explicitly specify the name of the key column using the ``on`` keyword
- which takes a column name or a list of column names
- this option works only if both the left and right ``DataFrame``s have the specified column name.

In [10]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

### The ``left_on`` and ``right_on`` keywords

To merge two datasets with different column names: 
-  ``left_on`` and ``right_on`` keywords to specify the two column names:

In [11]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]
                    })
display('df1', 
        'df3', 
        'pd.merge(df1, df3, left_on="employee", right_on="name")'
        )

The result has a redundant column that we can drop if desired–for example, by using the ``drop()`` method of ``DataFrame``s:

In [12]:
pd.merge(df1, 
         df3, 
         left_on="employee", 
         right_on="name"
         ).drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The ``left_index`` and ``right_index`` keywords

Rather than merging on a column, you would instead like to merge on an index.

For example, your data might look like this:

In [13]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


You can use the index as the key for merging by specifying the ``left_index`` and/or ``right_index`` flags in ``pd.merge()``
- If there are common columns to perform merge on
  -  left_on=None, right_on=None #None is defalut
- If the merge based on the index column, then
  - left_index=False, right_index=False #False is defalut

In [14]:
display('df1a', 
        'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


For convenience, ``DataFrame``s implement the ``join()`` method  
-  a merge that defaults to joining on indices

In [15]:
display('df1a', 'df2a', 
        'df1a.join(df2a)'
        )

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


If you'd like to mix `indices` and `columns`, you can combine ``left_index`` with ``right_on`` or ``left_on`` with ``right_index`` to get the desired behavior:

In [16]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

- All of these options also work with multiple indices and/or multiple columns
  - the interface for this behavior is very intuitive.

For more information on this, see the ["Merge, Join, and Concatenate" section](http://pandas.pydata.org/pandas-docs/stable/merging.html) of the Pandas documentation.

## Specifying Set Arithmetic for Joins

In all the preceding examples we have glossed over one important consideration in performing a join
  - the type of set arithmetic used in the join.

This comes up when a value appears in one key column but not the other. 

Consider this example:

In [17]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']
                    },
                   columns=['name', 'food']
                   )

df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']
                    },
                   columns=['name', 'drink']
                   )

In [18]:
display('df6', 'df7', 'pd.merge(df6, df7)')

### inner join

- Here we have merged two datasets that have only a single "name" entry in common: `Mary`.

- By `default`, the result contains the `intersection` of the two sets of inputs
  - this is what is known as an `inner join`
  - We can specify this explicitly using the ``how`` keyword, which defaults to ``"inner"``
  - Other options for the ``how`` keyword are ``'outer'``, ``'left'``, and ``'right'``
    - an `outer join` returns a join over the `union` of the input columns, and fills in all missing values with NAs
    - the `left join` and `right join` return joins over the left entries and right entries, respectively

In [19]:
pd.merge(df6, df7, 
         how='inner' #intersection
         )

,name,food,drink
0,Mary,bread,wine


In [20]:
display('df6', 'df7', 
        "pd.merge(df6, df7, how='outer')"#union
        )

In [21]:
display('df6', 'df7', 
        "pd.merge(df6, df7, how='left')"#all entries of Left table will be there
        )

In [22]:
display('df6', 'df7', 
        "pd.merge(df6, df7, how='right')"#all entries of Right table will be there
        )

All of these options can be applied straightforwardly to any of the preceding join types.

## Overlapping Column Names

### The ``suffixes`` Keyword

In the case of two input ``DataFrame``s have conflicting column names: 

In [23]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]
                    })

df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]
                    })

In [24]:
display('df8', 'df9', 
        'pd.merge(df8, df9, on="name")'
        )

-  the output would have two conflicting column names `rank`
-  the merge function automatically appends a suffix ``_x`` or ``_y`` to make the output columns unique

If these defaults are inappropriate, it is possible to specify a custom suffix using the ``suffixes`` keyword:

In [25]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

These suffixes work in any of the possible join patterns, and work also if there are multiple overlapping columns.

For more information on these patterns, see [Pandas "Merge, Join and Concatenate" documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html) for further discussion of these topics.

## Combining data from different sources

Merge and join operations come up most often when combining data from different sources.

In [26]:
pop     = pd.read_csv('https://raw.githubusercontent.com/tec03/Datasets/main/datasets/population.csv')
areas   = pd.read_csv('https://raw.githubusercontent.com/tec03/Datasets/main/datasets/US_area.csv')
abbrevs = pd.read_csv('https://raw.githubusercontent.com/tec03/Datasets/main/datasets/abbrv.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

Rank US states and territories by their 2010 population density.

- we'll have to combine the datasets to find the result
- start with a `many-to-one` merge that will give us the full state name within the population ``DataFrame``
- merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``
- use ``how='outer'`` to make sure `no data is thrown away` due to mismatched labels

In [27]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', 
                  right_on='abbreviation')
merged.head(2)

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL


In [28]:
merged = merged.drop('abbreviation', axis = 1) # drop duplicate info
merged.head(2)

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [29]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the ``population`` info is null; let's figure out which these are!

In [30]:
ppln_nulls = merged['population'].isnull()
ppln_nulls 

0       False
1       False
2       False
3       False
4       False
        ...  
2539    False
2540    False
2541    False
2542    False
2543    False
Name: population, Length: 2544, dtype: bool

In [31]:
merged[ppln_nulls].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


- It appears that all the null population values are from Puerto Rico prior to the year 2000
  - this is likely due to this data not being available from the original source

- Some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key

Let's figure out which regions lack this match:

In [32]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: 
  - the data includes entries for Puerto Rico (PR) and the United States as a whole (USA)
    - while these entries do not appear in the state abbreviation key.

We can fix these quickly by filling in appropriate entries. 

If it appears `PR` in the column `state/region` of the `merged` dataframe, place `Puerto Rico` in that row of the column `state` of the dataframe `merged`.  

In [33]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico' 

If it appears `USA` in the column `state/region` of the `merged` dataframe, place `United States` in that row of the column `state` of the dataframe `merged`.

In [34]:
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [35]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.

Examining our results, we will want to join on the ``state`` column in both

In [36]:
display('merged.head()', 'areas.head()')

In [37]:
final = pd.merge(merged,  #the merged df
                 areas,   #areas df 
                 on='state', 
                 how='left'
                 )
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Again, let's check for nulls to see if there were any mismatches:

In [38]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

There are nulls in the ``area`` column
-  we can take a look to see which regions were ignored here

In [39]:
area_nulls = final['area (sq. mi)'].isnull()
area_nulls

0       False
1       False
2       False
3       False
4       False
        ...  
2539     True
2540     True
2541     True
2542     True
2543     True
Name: area (sq. mi), Length: 2544, dtype: bool

In [40]:
final['state']

0             Alabama
1             Alabama
2             Alabama
3             Alabama
4             Alabama
            ...      
2539    United States
2540    United States
2541    United States
2542    United States
2543    United States
Name: state, Length: 2544, dtype: object

In [41]:
final['state'][area_nulls].head()

2496    United States
2497    United States
2498    United States
2499    United States
2500    United States
Name: state, dtype: object

In [42]:
final['state'][area_nulls].unique()

array(['United States'], dtype=object)

- We see that our ``areas`` ``DataFrame`` does not contain the area of the United States as a whole.
- We could insert the appropriate value (using the sum of all state areas, for instance), 
  - but in this case we'll just drop the null values because the population density of the entire United States is not relevant to our current discussion

In [43]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Now we have all the data we need. 

To answer the question of interest: 
- first select the portion of the data corresponding with the year 2000, and the total population
- use the ``query()`` function to do this quickly (this requires the ``numexpr`` package to be installed 

In [44]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Now let's compute the population density and display it in order.
- start by re-indexing our data on the state
-  then compute the result

In [45]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [46]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

- The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.

- We can see that by far the densest region in this dataset is  the District of Columbia; among states, the densest is New Jersey.

We can also check the end of the list:

In [47]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

- the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

<!--NAVIGATION-->
< [Prev](prev) | [toc](Index.ipynb) | [Next](next) >